<a id='top'></a>

#  IBM Q Replication Monitoring Jupyter Library
Author: Christian Lenke, IBM<br>
Version: 2019-07-23

## Table of Content

* [Preface](#intro)
* [Introduction](#intro)
* [Individual Setup](#setup)
<p>
* [Installation and import](#installation)
<p>
* [Function Definition](#commonfunc)
  * [Common functions](#commonfunc)
  * [Q Capture Functions](#capturefunc)
  * [Q Apply Functions](#applyfunc)

[Back to Top](#top)
<a id='intro'></a>

## Preface

This Jupyter Notebook is sample code. No warranty.

## Introduction

This set of Jupyter Notebook can be used to display status, performance, and health of an IBM Q Replication setup. The following Notebooks are provided:<br>
 <ul>
  <li>Q Capture Notebook</li>
  <li>Q Apply Notebook</li>
  <li>Q Replication function library Jupyter Notebook (this Notebook)</li>
</ul> 
This Notebook imports all required libraries and defines a library of Q Replication functions used by the other Q Replication Notebooks.

[Back to Top](#top)
<a id='setup'></a>

## Individual setup

### Customize the provided SQL files

Some of the provided SQL queries need customization:<br>
<b>qrep_check_qsubs_capture.sql (German version) or qrep_check_qsubs_capture_EN.sql (English version) :</b><br>
Search for <i><u>"change before execution"</u></i> and adjust the query depending on the platform of your Q Capture server (<u>z/OS: <i>datacapture</i> / LUW: <i>data_capture</i>)</u><br>
<b>qrep_check_qsubs_apply.sql (German version) or qrep_check_qsubs_apply_EN.sql (English version):</b><br>
Search for <i><u>"change before execution"</u></i> and adjust the name of your <i>Q Apply user</i> (to be able to check for the required database privileges). 2x in the query.<br>
Search for <i><u>"change before execution"</u></i> and adjust the query depending on the platform of your Q Apply server (<u>z/OS: <i>referencesauth</i> / LUW: <i>refauth</i></u>). 2x in the query.

### Customize the settings to reflect your environment

Adjust these 2 cells to your environment.

In [1]:
# Set the Db2 instance home - required to be able to connect to Db2
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\\Program Files\\IBM\\SQLLIB'

In [2]:
# Set the message language for the provided SQL queries:
#   English: 'EN'
#   German:  'DE'
msg_lang = 'EN'

# Set the path to the location where the monitoring SQL queries are stored
qrep_asnmonitor_file_path = 'C:\\1Lenke\\1WORK\\DB2UDB\\SQL\\QREPL\\asnmonitor'

# Set the path delimiter (comment either to Windows or to Linux/Unix):
# Windows
sql_file_delimiter = '\\'
# Linux/Unix
# sql_file_delimiter = '/'

Some config - don't change:

In [3]:
# Default language is EN
if msg_lang == 'EN':
    cust_msg_lang = '_EN'
elif msg_lang == 'DE':
    cust_msg_lang = ''
else:
    cust_msg_lang = '_EN'    

# constrution of file names for the specified language
qrep_check_qsubs_apply_file_name    = qrep_asnmonitor_file_path + sql_file_delimiter + \
                                      'qrep_check_qsubs_apply' + cust_msg_lang + '.sql'
qrep_check_qsubs_capture_file_name  = qrep_asnmonitor_file_path + sql_file_delimiter + \
                                      'qrep_check_qsubs_capture' + cust_msg_lang + '.sql'
qrep_monitor_apply_file_name        = qrep_asnmonitor_file_path + sql_file_delimiter + \
                                      'qrep_monitor_apply' + cust_msg_lang + '.sql'
qrep_monitor_capture_file_name      = qrep_asnmonitor_file_path + sql_file_delimiter + \
                                      'qrep_monitor_capture' + cust_msg_lang + '.sql'

[Back to Top](#top)
<a id='installation'></a>

## Installation

One time action to install the Db2 packages and other packages.

In [4]:
# Import the Anaconda packages missing in your installation.

# !pip install ipython-sql

# Install Db2 Driver
# Uncomment the following line for Db2 11.1 and Anaconda 2
# !pip install ibm_db

# Install Db2 Driver
# Uncomment the following line for Db2 11.1 and Anaconda 3
# !pip install "ibm_db==2.0.8a"

# Install SQL Alchemy
# !pip install ibm_db_sa

# Install qgrid which is required for Python Db2 Extensions (db2.iphynb)
#!pip install qgrid

# Install pixiedust which is required for Python Db2 Extensions (db2.iphynb)
#!pip install pixiedust

## Import

Import of all required libraries <br> Execute the cell in case you restarted the Anaconda kernel

In [5]:
import getpass

import ibm_db
import ibm_db_sa
import sqlalchemy
from sqlalchemy import *
from sqlalchemy import exc
from sqlalchemy.engine.url import URL
%load_ext sql

from datetime import date, datetime, timedelta

#import datetime
import pandas
import ibm_db_dbi
import json
import matplotlib
import matplotlib.pyplot as plt
import os
import pickle
import time
import sys
import re
import warnings
import qgrid
import numpy as np

import pixiedust

Pixiedust database opened successfully


[Back to Top](#top)
<a id='commonfunc'></a>

## Function definition

### Common functions

In [6]:
# Check parameters
def check_positive_int(varname, varvalue):
    if not isinstance(varvalue, int):
        print('ERROR: ' + varname + ' is not an integer')
    elif varvalue < 0:
         print('ERROR: ' + varname + ' is out of range (' + 
               str(varvalue) + '). Must be 0 or a positive INTEGER')
    else:
         print('INFO: ' + varname + ' checked ok')

def check_minus_1_or_positive_int(varname, varvalue):
    if not isinstance(varvalue, int):
        print('ERROR: ' + varname + ' is not an integer')
    elif varvalue < -1:
         print('ERROR: ' + varname + ' is out of range (' + 
               str(varvalue) + '). Must be -1 or a positive INTEGER')
    elif varvalue == 0:
         print('ERROR: ' + varname + ' is out of range (' + 
               str(varvalue) + '). Must be -1 or a positive INTEGER')            
    else:
         print('INFO: ' + varname + ' checked ok')            

In [7]:
# Connection to a database, returns the connection
def connect_to_db2(driver, user, password, host, port, alias):
    db2_con_url = driver + '://' + user + ':' + \
        password + '@' + host + ':' + port + '/' + alias
    db2_con = sqlalchemy.create_engine(db2_con_url, echo=False)
    return db2_con

In [8]:
# Calculation of monitoring data windows, depending on the parameters 
# defined in the Q Capture or Q Apply Notebooks
# Returns an array of 2 dates (start date and end date)

def calc_mon_start_end(cust_mon_start_date, cust_mon_end_date, dft_mon_date_range):
    
    # convert to negative for timedelta function
    dft_mon_date_range = dft_mon_date_range * -1
    
    if cust_mon_start_date == -1:
        # tmp_date = datetime.datetime.now() +  datetime.timedelta(days = dft_mon_date_range)
        tmp_date = datetime.now() + timedelta(days = dft_mon_date_range)
        calc_mon_start_date = (tmp_date.strftime("%Y-%m-%d-%H.%M.%S.%f"))
    else:
        calc_mon_start_date = cust_mon_start_date

    if cust_mon_end_date == -1:
        tmp_date = datetime.now() 
        calc_mon_end_date = (tmp_date.strftime("%Y-%m-%d-%H.%M.%S.%f"))
    else:
        calc_mon_end_date = cust_mon_end_date

    return [calc_mon_start_date,calc_mon_end_date]

In [9]:
# Setting the default colour coding for the SEVerity column. 
# Background colour and font colour 

def sev_background(value):
  """
  Background color in a dateframe:
  red for "ERROR", yellow for "WARNING",
  green for "INFO". Does not color NaN
  values.
  """

  if value == 'ERROR':
    background = 'red'
  elif value == 'WARNING':
    background = 'yellow'
  else:
    background = 'green'
    
  return 'background: %s' % background

def sev_foreground(value):
  """
  Font color in a dateframe:
  white for "ERROR", black for "WARNING",
  white for "INFO". Does not color NaN
  values.
  """

  if value == 'ERROR':
    color = 'white'
  elif value == 'INFO':
    color = 'white'
  else:
    color = 'black'
    
  return 'color: %s' % color

[Back to Top](#top)
<a id='capturefunc'></a>

### Q Capture functions

Q Capture Status Overview

In [10]:
# Function to read the query for the Q Capture status monitor from 
# file (qrep_monitor_capture.sql) and to execute the query. 
# Returns a data frame

def get_capture_status(conn):

    if not os.path.isfile(qrep_monitor_capture_file_name):
        print('File does not exist.')
    else:
        with open(qrep_monitor_capture_file_name , 'r') as f:
            sql_text_list = f.read()

    sql_text = ''.join(sql_text_list)        

    f.close()

    # debug
    # print(sql_text)
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    return df

In [26]:
# Function to read data from IBMQREP_SENDQUEUES 
# Returns a data frame

def get_sendq_state(conn):
    sql_text = "select 'Queues' as QUEUES, A.ACTIVE, I.INACTIVE from ( \
    (select 'NUM_QUEUES' as X, count(*) as active from ibmqrep_sendqueues where state = 'A')  A \
    inner join  \
    (select 'NUM_QUEUES' as X, count(*) as inactive from ibmqrep_sendqueues where state = 'I')  I \
    ON A.X = I.X );"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
        
    return df

In [12]:
# Function to read data from IBMQREP_SUBS
# Returns a data frame

def get_substate_by_sendq(conn):
    sql_text = "select A.SENDQ as SENDQ, coalesce(A.ACTIVE , 0) as ACTIVE, coalesce(I.INACTIVE , 0) as INACTIVE \
    from ( \
    (select SENDQ, count(*) as active from ibmqrep_subs where state = 'A' group by sendq)  A  \
    full outer join   \
    (select SENDQ, count(*) as inactive from ibmqrep_subs where state = 'I' group by sendq)  I  \
    ON A.SENDQ = I.SENDQ \
    );"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

Q Capture Log Reader Performance Statistics - CAPMON

In [13]:
# Function to read data from IBMQREP_CAPMON
# Returns a data frame

def get_perf_logrd(conn, range):

    # Default: testschema = ''  to read from default schema
    # Set it to a specific test schema only for test purposes
    testschema = ''
    # testschema = 'DB2R_20190509.'

    if range == -1:
        filterclause = ""
    else:
        filterclause = "where monitor_time > (select max(monitor_time) - " + str(range) + " days \
                        from " + testschema + "ibmqrep_capmon ) "
        
    sql_text = "select varchar(date(monitor_time)) concat '-' \
            concat varchar(hour(monitor_time)) concat ':00' as monitor_date, \
            sum(rows_processed) as sum_rows_processed, \
            dec(max(current_memory)) / 1024 / 1024 as max_memory_mb, \
            sum(trans_spilled) as sum_trans_spilled \
            from " + testschema + "ibmqrep_capmon " + filterclause + \
            " group by date(monitor_time), hour(monitor_time) \
            order by date(monitor_time), hour(monitor_time)"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    return df

In [14]:
# Function to create a figure to display the following values of IBMQREP_CAPMON:
#  - CURRENT_MEMORY
#  - ROWS_PROCESSED
#  - TRANS_SPILLED
# Returns the plot

def plot_capmon(df_logr,numdays):

    fig = plt.figure(figsize=(20,10))
    
    if numdays == -1:
        limittext = ' - All available data'
    else:
        limittext = ' - Last ' + str(numdays) + ' days of CAPMON data'

    # First axes: sum_rows_processed
    ax1 = fig.add_subplot(111,title='Capture log reader throughput and memory per hour')

    # the ax keyword sets the axis that the data frame plots to
    df_logr.plot(ax=ax1, x='monitor_date', y='sum_rows_processed',
                 kind='line', linewidth=1, linestyle='-', color='m', 
                 title='CAPMON statistics ' + limittext)
    
    plt.ticklabel_format(style='plain', axis='y')
    plt.xticks(df_logr.index,df_logr['monitor_date'].values,rotation=70)

    xticks1 = plt.gca().xaxis.get_major_ticks()

    for j in range(len(xticks1)):
        if (numdays > 5) or (numdays == -1):
            if j % 24 != 0:
                xticks1[j].set_visible(False)
            else:
                xticks1[j].set_visible(True)
        elif numdays > 2:                    
            if j % 4 != 0:
                xticks1[j].set_visible(False)
            else:
                xticks1[j].set_visible(True)
        else:
            if j % 2 != 0:
                xticks1[j].set_visible(False)
            else:
                xticks1[j].set_visible(True)                

    ax1.set_xlabel('monitor date (yyyy-mm-dd-hh:mm)')
    ax1.set_ylabel('rows processed per hour')

    # Second axes: max_memory_mb    
    ax2 = ax1.twinx()

    df_logr.plot(ax=ax2, x='monitor_date', y='max_memory_mb', 
                kind='line', linewidth=1, linestyle=':', color='b')

    ax2.set_ylabel('max memory (MB)')
    plt.legend(loc='upper left')

    # Third axes: trans_spilled
    ax3 = ax1.twinx()
    ax3.spines["right"].set_position(("axes", 1.1))

    if max(df_logr['sum_trans_spilled']) == 0: 
        maxtsp = 5
    else:
        maxtsp = max(df_logr['sum_trans_spilled'])

    ax3.set_ylim(0, maxtsp)

    df_logr.plot(ax=ax3, x='monitor_date', y='sum_trans_spilled', 
                kind='bar', legend=False)

    ax3.set_ylabel('monster trans')
    
    return(plt)

Q Capture Publisher Performance Statistics - CAPQMON

In [15]:
# Function to read data from IBMQREP_CAPQMON
# Returns a data frame
    
def get_perf_publ(conn, range):
    
    # Default: testschema = ''  to read from default schema
    # Set it to a specific test schema only for test purposes
    testschema = ''
    # testschema = 'DB2R_20190509.'

    if range == -1:
        filterclause = ""
    else:
        filterclause = "where monitor_time > (select max(monitor_time) - " + str(range) + " days \
                        from " + testschema + "ibmqrep_capmon ) "  
    
    sql_text = "select varchar(date(monitor_time)) concat '-' \
            concat varchar(hour(monitor_time)) concat ':00' as monitor_date, \
            sendq, \
            sum(rows_published) as sum_rows_published, \
            sum(trans_published) as sum_trans_published, \
            sum(mq_messages) as sum_mq_messages, \
            max(xmitqdepth) as max_xmitqdepth \
            from " + testschema + "ibmqrep_capqmon " + filterclause + \
            " group by sendq, date(monitor_time), hour(monitor_time) \
            order by date(monitor_time), hour(monitor_time)"

    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    return df

In [16]:
# Todo: xticks only for final plot. Legend misplaced.

# Function to create a figure with n plots (one lot per receive queue) to display the 
# following values of IBMQREP_CAPQMON:
#  - SUM_ROWS_PUBLISHED
#  - SUM_MQ_MESSAGES
#  - XMITQDEPTH
# Returns the plots

def plot_capqmon(df_publ_allq,qs,numdays):

    # The hight of the following figures depends on the number of distinct send queues.
    # The more queues, the less space per individual queue (to limit the size of the figure)
    if numqs >=4:
        calc_fig_height = numqs * 6
    if numqs >=2:
        calc_fig_height = numqs * 8
    else:
        calc_fig_height = numqs * 10
    
    fig3 = plt.figure(1,figsize=(20,calc_fig_height))

    if numdays == -1:
        limittext = ' - All available data'
    else:
        limittext = ' - Last ' + str(numdays) + ' days of CAPQMON data'
        
    # get a 2-dimensional object
    ax = {}
    
    # print('DEBUG: qs=' + str(qs))
    
    # Loop, for each reqeive queue
    for i in range(0,qs):
        # print('DEBUG: i=' + str(i) + '; ' + df_publ_allq.sendq.unique()[i])

        # First axes: sum_rows_published,sum_mq_messages

        # Axis position
        #  First parameter: Number of plots = number of queues
        #  Second parameter: Horizintal position or column (always 1 = only 1 column)
        #  Third parameter: Vertical position or row (1, 2, 3, ... depending on the loop)
        ax[i,0] = fig3.add_subplot(qs,1,i+1)

        # Plot. Data frame limited to one queue per iteration
        df_publ_allq[df_publ_allq.sendq == df_publ_allq.sendq.unique()[i]].plot(ax=ax[i,0], 
            x='monitor_date', 
            y=['sum_rows_published','sum_mq_messages'],
            kind='line', linewidth=1, linestyle='-', 
            title=df_publ_allq.sendq.unique()[i] + ' APPLYMON statistics' + limittext)
        
        # monitor_date as xticks
        plt.xticks(df_publ_allq.index,df_publ_allq['monitor_date'].values,rotation=70)

        xticks1 = plt.gca().xaxis.get_major_ticks()

        # setting the tick density        
        for j in range(len(xticks1)):
            if (numdays > 5) or (numdays == -1):
                if j % 24 != 0:
                    xticks1[j].set_visible(False)
                else:
                    xticks1[j].set_visible(True)
            elif numdays > 2:                    
                if j % 4 != 0:
                    xticks1[j].set_visible(False)
                else:
                    xticks1[j].set_visible(True)
            else:
                if j % 2 != 0:
                    xticks1[j].set_visible(False)
                else:
                    xticks1[j].set_visible(True)                
    
        ax[i,0].set_xlabel('monitor date (yyyy-mm-dd-hh:mm)')
        ax[i,0].set_ylabel('Number of')

        # Second axes: Rows processed
        
        ax[i,1] = ax[i,0].twinx()

        # Plot. Data frame limited to one queue per iteration        
        df_publ_allq[df_publ_allq.sendq == df_publ_allq.sendq.unique()[i]].plot(ax=ax[i,1], 
            x='monitor_date', 
            y='max_xmitqdepth', 
            kind='line', linewidth=1, linestyle='-', color='m')

        plt.ticklabel_format(style='plain', axis='y')
        ax[i,1].set_ylabel('Max XMIT Queue Depth')
        plt.legend(loc='upper left')

    return(plt)

Q Capture Queue and Sub Details

In [17]:
# Function to read data from IBMQREP_SENDQUEUES
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_sendq_details(conn, schema):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_SENDQUEUES' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
        
    sql_text = "SELECT " + collist + " FROM IBMQREP_SENDQUEUES order by sendq;"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [18]:
# Function to read data from IBMQREP_SUBS
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_subs_details(conn, schema):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_SUBS' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_SUBS order by subname;"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

Q Capture Anomalies

In [19]:
# Function to read the query for the Q Subscription check from 
# file (qrep_check_subs_capture.sql) and to execute the query. 
# Returns a data frame

def get_capture_anomylies(conn):

    if not os.path.isfile(qrep_check_qsubs_capture_file_name):
        print('File does not exist.')
    else:
        with open(qrep_check_qsubs_capture_file_name , 'r') as f:
            sql_text_list = f.read()

    sql_text = ''.join(sql_text_list)        

    f.close()

    # debug
    # print(sql_text)
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    return df

Q Capture Control Table Details

In [20]:
# Function to read data from IBMQREP_CAPPARMS
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_capparms(conn, schema):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_CAPPARMS' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_CAPPARMS;"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [23]:
# Function to read data from IBMQREP_CAPTRACE (only numrows)
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_captrace(conn, schema, numrows):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_CAPTRACE' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
            
    sql_text = "SELECT " + collist + " FROM ibmqrep_captrace order by trace_time desc "

    if numrows > 0:
        sql_text = sql_text + " fetch first " + str(numrows) + " rows only;"
    else:
        sql_text = sql_text + ";"
        
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [24]:
# Function to read data from IBMQREP_CAPMON with a WHERE clause respecting the 
# calculated data range. Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_capmon(conn, schema, start_date, end_date):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_CAPMON' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    listOfHexCols = ['RESTART_SEQ' , 'CURRENT_SEQ', 'RESTART_MAXCMTSEQ']
    for index, row in cat_df.iterrows():
        if row['name'] in listOfHexCols:
            colexp = "hex(" + row['name'] + ") as " + row['name']
        else: 
            colexp = row['name']

        if index == 0:
            collist = colexp
        else:
            collist = collist + ', ' + colexp
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_CAPMON \
                where monitor_time >= '" + start_date + "' \
                  and  monitor_time <= '" + end_date + "' \
                order by monitor_time desc;"    
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [ ]:
# Function to read data from IBMQREP_CAPQMON with a WHERE clause respecting the 
# calculated data range. Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_capqmon(conn, schema, start_date, end_date):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_CAPQMON' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    listOfHexCols = ['CURRENT_SEQ' , 'RESTART_SEQ', 'RESTART_MAXCMTSEQ']
    for index, row in cat_df.iterrows():
        if row['name'] in listOfHexCols:
            colexp = "hex(" + row['name'] + ") as " + row['name']
        else: 
            colexp = row['name']

        if index == 0:
            collist = colexp
        else:
            collist = collist + ', ' + colexp
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_CAPQMON \
                where monitor_time >= '" + start_date + "' \
                  and  monitor_time <= '" + end_date + "' \
                order by monitor_time desc;"   
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

[Back to Top](#top)
<a id='applyfunc'></a>

### Q Apply functions

Q Apply status overview

In [ ]:
# Function to read the query for the Q Apply status monitor from 
# file (qrep_monitor_apply.sql) and to execute the query. 
# Returns a data frame

def get_apply_status(conn):

    if not os.path.isfile(qrep_monitor_apply_file_name):
        print('File does not exist.')
    else:
        with open(qrep_monitor_apply_file_name , 'r') as f:
            sql_text_list = f.read()

    sql_text = ''.join(sql_text_list)        

    f.close()

    # debug
    # print(sql_text)
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    return df

In [ ]:
# Function to read data from IBMQREP_RECVQUEUES 
# Returns a data frame

def get_recvq_state(conn):
    sql_text = "select 'Queues' as QUEUES, A.ACTIVE, I.INACTIVE from ( \
    (select 'NUM_QUEUES' as X, count(*) as active from ibmqrep_recvqueues where state = 'A')  A \
    inner join  \
    (select 'NUM_QUEUES' as X, count(*) as inactive from ibmqrep_recvqueues where state = 'I')  I \
    ON A.X = I.X );"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
        
    return df

In [ ]:
# Function to read data from IBMQREP_TARGETS
# Returns a data frame

def get_substate_by_recvq(conn):
    sql_text = "select A.RECVQ as RECVQ, coalesce(A.ACTIVE , 0) as ACTIVE, coalesce(I.INACTIVE , 0) as INACTIVE \
    from ( \
    (select RECVQ, count(*) as active from ibmqrep_targets where state = 'A' group by recvq)  A  \
    full outer join   \
    (select RECVQ, count(*) as inactive from ibmqrep_targets where state = 'I' group by recvq)  I  \
    ON A.RECVQ = I.RECVQ \
    );"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

Q Apply Subscription type by RECVQ

In [ ]:
# Function to read different subscription types from IBMQREP_TARGETS
# Returns a data frame

def get_subtype_by_recvq(conn):
    sql_text = "select recvq, \
         case subtype \
           when 'B' then 'BIDI' \
           when 'P' then 'P2P' \
           when 'U' then \
             case target_type \
               when 1 then 'USER TABLE' \
               when 5 then 'STOREDPROC' \
               when 2 then  \
                 case \
                   when CCD_CONDENSED = 'Y' and CCD_COMPLETE = 'Y' then \
                     'COND/COMP CCD' \
                   when CCD_CONDENSED = 'N' and CCD_COMPLETE = 'N' then \
                     'NON-COND/NON-COMP CCD' \
                   when CCD_CONDENSED = 'Y' and CCD_COMPLETE = 'N' then \
                     'COND/NON-COMP CCD' \
                   when CCD_CONDENSED = 'N' and CCD_COMPLETE = 'Y' then \
                     'NON-COND/COMP CCD' \
                   else 'OTHER CCD' \
                 end \
               else 'OTHER TARGET TYPE' \
             end \
           else 'OTHER SUBTYPE' \
         end as calc_type, \
         count(*) as count_type  \
         from lsn.ibmqrep_targets  \
         group by recvq, subtype, target_type, ccd_condensed, ccd_complete;"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [ ]:
# Function to create a figure with n donut plots (one lot per receive queue) to display the 
# subscription types by receive queue:

def plot_types(df_st,numqs,figrows):

    # The hight of the following figures depends on the number of distinct receive queues.
    # The more queues, the less space per individual queue (to limit the size of the figure)
    
    if numqs > 10:
        calc_fig_height = numqs * 2
    elif numqs > 2:
        calc_fig_height = numqs * 4
    else:
        calc_fig_height = numqs * 10
    
    # rows = number of receive queues / 2; columns always 2
    figpie, axs = plt.subplots(figrows,2)

    figpie.set_figheight(calc_fig_height)
    figpie.set_figwidth(20)

    # create a white circle to later convert the pie into a donut
    n, radii = 50, [0, .5]
    theta = np.linspace(0, 2*np.pi, n, endpoint=True)
    xs = np.outer(radii, np.cos(theta))
    ys = np.outer(radii, np.sin(theta))
    xs[1,:] = xs[1,::-1]
    ys[1,:] = ys[1,::-1]

    # Loop, plot a distinct pie for each receive queue 
    for i in range(0,numqs):
        
        # reset the colors list for each queue. The colors have to be set for each queue
        # individually, to be able to assign the same color to always the same subscription 
        # type. Example: bidi always blue
        mycolors = []
       
        # current row = current queue devided by 2 as int - example int(5 / 2) = int(2.5) = 2
        row = int(i / 2)
        # current column = current queue modulo 2 - example: 5 % 2 = 1
        col = i % 2
        
        # store the name of the current recvq
        eval_recvq = df_st.recvq.unique()[i]

        # Filter, only current RECVQ
        df_st_pl = df_st[df_st.recvq == eval_recvq]
    
        # All distinct subscription types for the current recvq
        qtypes = df_st_pl.calc_type.unique()
    
        # Loop through all types of that queue and assign consistent colors,
        # same color for same type in all pies
        for c in range(0,len(qtypes)):

            # print(mycolors)
            # print(str(c), qtypes[c])
            if qtypes[c] == 'BIDI':
                mycolors.append('tab:blue')
            elif qtypes[c] == 'P2P': 
                mycolors.append('tab:cyan')
            elif qtypes[c] == 'USER TABLE': 
                mycolors.append('tab:green')
            elif qtypes[c] == 'STOREDPROC': 
                mycolors.append('tab:red')
            elif qtypes[c] == 'COND/COMP CCD': 
                mycolors.append('tab:pink')
            elif qtypes[c] == 'NON-COND/NON-COMP CCD': 
                mycolors.append('tab:orange')
            elif qtypes[c] == 'COND/NON-COMP CCD': 
                mycolors.append('tab:olive')
            elif qtypes[c] == 'NON-COND/COMP CCD': 
                mycolors.append('tab:grey')
            elif qtypes[c] == 'OTHER CCD': 
                mycolors.append('orangered')
            elif qtypes[c] == 'OTHER TARGET TYPE': 
                mycolors.append('orangered')
            else: 
                mycolors.append('orangered')
            # print(str(c), qtypes[c], mycolors[c])
            
        # get a total of all subs across all types. Required to convert relative shares to absolute shares
        total = sum(df_st_pl['count_type'])
    
        axs[row,col].axis('equal') 
    
        # plot pie, with absolute values
        axs[row,col].pie(df_st_pl['count_type'], labels=df_st_pl['calc_type'], colors=mycolors, 
                         autopct=lambda p: '{:.0f}'.format(p * total / 100), pctdistance=0.7, startangle=90, 
                         textprops={'fontsize': 14, 'color' : 'black'})
    
        # convert pie into a donut 
        axs[row,col].fill(np.ravel(xs), np.ravel(ys), edgecolor='white', color='white')
        axs[row,col].set_title(eval_recvq, fontdict={'fontsize' : 16, 'fontweight' : 'bold'}) 
        
    return(plt)

Q Apply Performance Statistics

In [ ]:
# Function to read data from IBMQREP_APPLYMON, GROUPed BY HOUR(MONITOR_DATE)
# Returns a data frame

def get_perf_applymon(conn, range):

    # Default: testschema = ''  to read from default schema
    # Set it to a specific test schema only for test purposes
    testschema = ''
    # testschema = 'IDH1I1A_20190509.'
    
    if range == -1:
        filterclause = ""
    else:
        filterclause = "where monitor_time > (select max(monitor_time) - " + str(range) + " days \
                        from " + testschema + "ibmqrep_applymon ) "
            
    sql_text = "select varchar(date(monitor_time)) concat '-' \
            concat right(digits(hour(monitor_time)) , 2) concat ':00' as monitor_date, \
            recvq, \
            dec(max(capture_latency)) / 1000 as max_capture_latency_sec, \
            dec(max(qlatency)) / 1000 as max_qlatency_sec, \
            dec(max(apply_latency)) / 1000 as max_apply_latency_sec, \
            sum(rows_processed) as sum_rows_processed, \
            max(qdepth) as max_qdepth \
            from " + testschema + "ibmqrep_applymon " + filterclause + \
            " group by recvq, date(monitor_time), hour(monitor_time) \
            order by date(monitor_time), hour(monitor_time), recvq;"
   
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex)) 
    return df

In [ ]:
# Todo: xticks only for final plot. Legend misplaced.

# Function to create a figure with n plots (one lot per receive queue) to display the 
# following values of IBMQREP_APPLYMON:
#  - CAPTURE_LATENCY
#  - APPLY_LATENCY
#  - QLATENCY
#  - ROWS_PROCESSED
# Returns the plots

def plot_lat(lat_df_allq,qs,numdays,grouping):

    # The hight of the following figures depends on the number of distinct receive queues.
    # The more queues, the less space per individual queue (to limit the size of the figure)
    if numqs >=4:
        calc_fig_height = numqs * 6
    if numqs >=2:
        calc_fig_height = numqs * 8
    else:
        calc_fig_height = numqs * 10
    
    fig3 = plt.figure(1,figsize=(20,calc_fig_height))

    if numdays == -1:
        limittext = ' - All available data'
    else:
        limittext = ' - Last ' + str(numdays) + ' days of APPLYMON data'
        
    # get a 2-dimensional object
    ax = {}
    
    # print('DEBUG: qs=' + str(qs))
    
    # Loop, for each reqeive queue
    for i in range(0,qs):
        # print('DEBUG: i=' + str(i) + '; ' + lat_df_allq.recvq.unique()[i])

        # First axes: Latency
        
        # old:
        # plotpos = qs * 100 + 11 + i
        # print('plotpos=' + str(plotpos))
        # ax[i,0] = fig3.add_subplot(plotpos,title=lat_df_allq.recvq.unique()[i] + ' - Latency and throughput per hour')

        # new:
        # Axis position
        #  First parameter: Number of plots = number of queues
        #  Second parameter: Horizintal position or column (always 1 = only 1 column)
        #  Third parameter: Vertical position or row (1, 2, 3, ... depending on the loop)
        ax[i,0] = fig3.add_subplot(qs,1,i+1)

        # Plot. Data frame limited to one queue per iteration
        lat_df_allq[lat_df_allq.recvq == lat_df_allq.recvq.unique()[i]].plot(ax=ax[i,0], 
            x='monitor_date', 
            y=['max_apply_latency_sec','max_qlatency_sec','max_capture_latency_sec'],
            kind='area', title=lat_df_allq.recvq.unique()[i] + ' APPLYMON statistics' + limittext)
        
        # monitor_date as xticks
        plt.xticks(lat_df_allq.index,lat_df_allq['monitor_date'].values,rotation=70)

        xticks1 = plt.gca().xaxis.get_major_ticks()
        
        # setting the tick density
        if grouping == 'by_hour':
            if (numdays > 5) or (numdays == -1):
                if len(xticks1) < 100:
                    tick_devider = 24
                else:
                    tick_devider = 48                
            elif numdays > 2: 
                tick_devider = 4
            else: 
                tick_devider = 2
        else: 
            # by_day or by_monitor_interval
            if len(xticks1) < 30:
                tick_devider = 31
            if len(xticks1) < 100:
                tick_devider = 2
            else:
                tick_devider = 10
        
        for j in range(len(xticks1)):
            if j % tick_devider != 0:
                xticks1[j].set_visible(False)
    
        ax[i,0].set_xlabel('monitor date (yyyy-mm-dd-hh:mm)')
        ax[i,0].set_ylabel('seconds')
        ax[i,0].legend(loc='upper left')        
        
        # Second axes: Rows processed
        ax[i,1] = ax[i,0].twinx()

        # Plot. Data frame limited to one queue per iteration        
        lat_df_allq[lat_df_allq.recvq == lat_df_allq.recvq.unique()[i]].plot(ax=ax[i,1], 
            x='monitor_date', 
            y='sum_rows_processed', 
            kind='line', linewidth=2, linestyle='-', color='m')

        plt.ticklabel_format(style='plain', axis='y')

        if grouping == 'by_hour':
            ax[i,1].set_ylabel('rows processed per hour')
        elif grouping == 'by_day':            
            ax[i,1].set_ylabel('rows processed per day')
        else:
            ax[i,1].set_ylabel('rows processed')        
            
        ax[i,1].legend(loc='upper right')        

    return(plt)

In [ ]:
# Function to read data from IBMQREP_APPLYMON, GROUPed BY DATE(MONITOR_DATE)
# Returns a data frame

def get_perf_applymon_day(conn):

    # Default: testschema = ''  to read from default schema
    # Set it to a specific test schema only for test purposes
    testschema = ''
    # testschema = 'IDH1I1A_20190509.'
    
    sql_text = "select varchar(date(monitor_time)) as monitor_date, \
            recvq, \
            dec(max(capture_latency)) / 1000 as max_capture_latency_sec, \
            dec(max(qlatency)) / 1000 as max_qlatency_sec, \
            dec(max(apply_latency)) / 1000 as max_apply_latency_sec, \
            sum(rows_processed) as sum_rows_processed, \
            max(qdepth) as max_qdepth \
            from " + testschema + "ibmqrep_applymon \
            group by recvq, date(monitor_time) \
            order by date(monitor_time), recvq;"
   
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex)) 
    return df

In [ ]:
# Function to read data from IBMQREP_APPLYMON, ungrouped
# Returns a data frame

def get_perf_applymon_detail(conn, start_ts):

    # Default: testschema = ''  to read from default schema
    # Set it to a specific test schema only for test purposes
    testschema = ''
    # testschema = 'BITMARCK_20190624.'
    
    if start_ts == 'max':
        filterclause = "WHERE MONITOR_TIME > (select max(monitor_time) - 1 hour \
                        from " + testschema + "ibmqrep_applymon)"
    else:
        filterclause = "WHERE MONITOR_TIME > timestamp('" + start_ts + "') \
                       AND MONITOR_TIME <= timestamp('" + start_ts + "') + 1 hour "
    
    sql_text = "select m.monitor_time, \
        m.recvq, \
        dec(m.capture_latency) / 1000 as capture_latency_sec, \
        dec(m.qlatency) / 1000 as qlatency_sec, \
        dec(m.apply_latency) / 1000 as apply_latency_sec, \
        dec(m.rows_processed) / (select dec(p.monitor_interval) / 1000 \
        from " + testschema + "ibmqrep_applyparms p ) as rows_processed_sec, \
        m.qdepth, \
        dec(m.mqget_time) / 1000 as mqget_time_sec, \
        dec(m.apply_sleep_time) / r.num_apply_agents / 1000 as apply_sleep_time_agent_sec, \
        dec(m.workq_wait_time) / 1000 as workq_wait_time_sec, \
        dec(m.dbms_time) / 1000 as dbms_time_sec, \
        dec(m.dependency_delay) / 1000 as  dependency_delay_sec\
        from " + testschema + "ibmqrep_applymon m, " + testschema + "ibmqrep_recvqueues r \
        " + filterclause + \
        " and m.recvq = r.recvq \
        order by m.monitor_time, m.recvq;"

    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    return df

In [ ]:
# Todo: xticks only for final plot. Legend misplaced.

# Function to create a figure with n plots (one lot per receive queue) to display the 
# following values of IBMQREP_APPLYMON:
#  - CAPTURE_LATENCY
#  - APPLY_LATENCY
#  - QLATENCY
#  - ROWS_PROCESSED
# Returns the plots

def plot_lat_details(lat_df_allq,qs,start_ts,grouping):

    if qs >=4:
        calc_fig_height = numqs * 6
    if qs >=2:
        calc_fig_height = numqs * 8
    else:
        calc_fig_height = numqs * 10

    
    fig3 = plt.figure(1,figsize=(20,calc_fig_height))

    limittext = " - One hour of available data starting from '" + start_ts + "'"
        
    # get a 2-dimensional object
    ax = {}
    
    # print('DEBUG: qs=' + str(qs))
    
    # Loop, for each reqeive queue
    for i in range(0,qs):
        # print('DEBUG: i=' + str(i) + '; ' + lat_df_allq.recvq.unique()[i])

        # First axes: Latency
        
        # old:
        # plotpos = qs * 100 + 11 + i
        # print('plotpos=' + str(plotpos))
        # ax[i,0] = fig3.add_subplot(plotpos,title=lat_df_allq.recvq.unique()[i] + ' - Latency and throughput per hour')

        # new:
        # Axis position
        #  First parameter: Number of plots = number of queues
        #  Second parameter: Horizintal position or column (always 1 = only 1 column)
        #  Third parameter: Vertical position or row (1, 2, 3, ... depending on the loop)
        ax[i,0] = fig3.add_subplot(qs,1,i+1)

        # Plot. Data frame limited to one queue per iteration
        lat_df_allq[lat_df_allq.recvq == lat_df_allq.recvq.unique()[i]].plot(ax=ax[i,0], 
            x='monitor_time', 
            y=['apply_latency_sec','qlatency_sec','capture_latency_sec'],
            kind='area', title=lat_df_allq.recvq.unique()[i] + ' APPLYMON statistics' + limittext)
        
        # monitor_time as xticks
        plt.xticks(lat_df_allq.index,lat_df_allq['monitor_time'].values,rotation=70)

        # only display 1 tick per day
        xticks1 = plt.gca().xaxis.get_major_ticks()

        if grouping == 'by_hour':
            if (numdays > 5) or (numdays == -1):
                if len(xticks1) < 100:
                    tick_devider = 24
                else:
                    tick_devider = 48                
            elif numdays > 2: 
                tick_devider = 4
            else: 
                tick_devider = 2
        else: 
            # by day or by_monitor_interval
            if len(xticks1) < 30:
                tick_devider = 31
            if len(xticks1) < 100:
                tick_devider = 2
            else:
                tick_devider = 10
        
        for j in range(len(xticks1)):
            if j % tick_devider != 0:
                xticks1[j].set_visible(False)
    
        ax[i,0].set_xlabel('monitor time (yyyy-mm-dd-hh:mm:ss)')
        ax[i,0].set_ylabel('seconds')
        ax[i,0].legend(loc='upper left')

        # Second axes: Rows processed
        ax[i,1] = ax[i,0].twinx()

        # Plot. Data frame limited to one queue per iteration        
        lat_df_allq[lat_df_allq.recvq == lat_df_allq.recvq.unique()[i]].plot(ax=ax[i,1], 
            x='monitor_time', 
            y=['rows_processed_sec','qdepth'], 
            kind='line', linewidth=2, linestyle='-', color=['magenta','slategrey'])

        plt.ticklabel_format(style='plain', axis='y')
        # ax[i,1].set_ylabel('rows processed per second')
        ax[i,1].legend(loc='upper right')        
        
        # Third axes: timimg
        ax[i,2] = ax[i,0].twinx()
        ax[i,2].spines["right"].set_position(("axes", 1.1))

        # Plot. Data frame limited to one queue per iteration        
        lat_df_allq[lat_df_allq.recvq == lat_df_allq.recvq.unique()[i]].plot(ax=ax[i,2], 
            x='monitor_time', 
            y=['mqget_time_sec','apply_sleep_time_agent_sec','workq_wait_time_sec','dependency_delay_sec'],
            kind='line', linewidth=2, linestyle='--', color=['deepskyblue','orangered','indigo','yellow'])

        plt.ticklabel_format(style='plain', axis='y')
        ax[i,2].set_ylabel('seconds')
        ax[i,2].legend(loc='center right')                

    return(plt)

Q Apply Queue and Sub Details

In [ ]:
# Function to read data from IBMQREP_RECVQUEUES
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_recvq_details(conn, schema):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_RECVQUEUES' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
        
    sql_text = "SELECT " + collist + " FROM IBMQREP_RECVQUEUES order by recvq;"

    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [ ]:
# Function to read data from IBMQREP_TARGETS
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_targets_details(conn, schema):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_TARGETS' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_TARGETS order by subname;"    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

Q Apply Anomalies

In [ ]:
# Function to read the query for the Q Subscription check from 
# file (qrep_check_subs_apply.sql) and to execute the query. 
# Returns a data frame

def get_apply_anomylies(conn):

    if not os.path.isfile(qrep_check_qsubs_apply_file_name):
        print('File does not exist.')
    else:
        with open(qrep_check_qsubs_apply_file_name , 'r') as f:
            sql_text_list = f.read()

    sql_text = ''.join(sql_text_list)        

    f.close()

    # debug
    # print(sql_text)
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    return df

Q Apply Control Table Details

In [ ]:
# Function to read data from IBMQREP_APPLYPARMS
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_applyparms(conn, schema):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_APPLYPARMS' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_APPLYPARMS;"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [ ]:
# Function to read data from IBMQREP_APPLYTRACE (only numrows)
# Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_applytrace(conn, schema, numrows):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_APPLYTRACE' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    for index, row in cat_df.iterrows():
        if index == 0:
            collist = row['name']
        else:
            collist = collist + ', ' + row['name']
            
    sql_text = "SELECT " + collist + " FROM ibmqrep_applytrace order by trace_time desc "

    
    if numrows > 0:
        sql_text = sql_text + " fetch first " + str(numrows) + " rows only;"

    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

In [ ]:
# Function to read data from IBMQREP_APPLYMON with a WHERE clause respecting the 
# calculated data range. Returns a data frame
# The column list is not hard coded but retrieved from the catalog (all columns) to more flexibly
# react on new control table columns

def get_applymon(conn, schema, start_date, end_date):
    cat_stmt = "select name from sysibm.syscolumns \
                where tbcreator = '" + schema + "' \
                  and tbname = 'IBMQREP_APPLYMON' \
                order by COLNO;"
    
    try:
        cat_df = pandas.read_sql_query(cat_stmt, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))

    listOfHexCols = ['OLDEST_COMMIT_LSN' , 'OLDEST_COMMIT_SEQ']
    for index, row in cat_df.iterrows():
        if row['name'] in listOfHexCols:
            colexp = "hex(" + row['name'] + ") as " + row['name']
        else: 
            colexp = row['name']

        if index == 0:
            collist = colexp
        else:
            collist = collist + ', ' + colexp
            
    sql_text = "SELECT " + collist + " FROM IBMQREP_APPLYMON \
                where monitor_time >= '" + start_date + "' \
                  and  monitor_time <= '" + end_date + "' \
                order by monitor_time desc;"
    
    try:
        df = pandas.read_sql_query(sql_text, conn)
    except exc.SQLAlchemyError as ex:
        print('We got a problem......:' + str(ex))
    
    return df

### Final Message

In [ ]:
print("IBM Q Replication Monitoring Jupyter Library.ipynb loaded successfully")

## Release Notes

<b>02.12.2019:</b> Global fix: 'connection' replaced by 'conn' for all invocations of read_sql_query(sql_text, conn).<br>
<b>02.12.2019:</b> Some functions (e.g., 'get_sendq_details') just retrieve the content of IBMQREP control tables. Previously, these SQL queries contained a list of columns (all columns). This was changed for multiple queries to first generate the list of columns from the Db2 catalog and then build and execute the query. This increases the robustness of the Notebook because with that feature it supports multiple control table architecture levels (prevents errors because of missing (optional) columns).<p>
<b>02.12.2019:</b>All "print('We got a problem......:' + str(ex))" changed to "print('We got a problem......:' + str(ex))" // ex cannot be concatenated to string